In [1]:
import tweepy

In [3]:
# Twitter API credentials
consumer_key = 'F5bwjzfNDf7LWlX0VbHeMqBQH'
consumer_secret = 'aRv5Kxob0mta5tGbiW48cdxi8GdB12woo8bnIsNmMTgJ2Ma3ky'
access_token = '1194164288132247552-zoWxVkGJ2MWfNW8HAkDIP6bPb6lpQC'
access_token_secret = 'chFGTNtFh0Bdzpxr6TvXuZiHr4tjvj114TD3fFvpNQzLz'

# Authenticate
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True)

def fetch_tweets(username, num_tweets=200):
    tweets = []
    try:
        # Collect tweets
        for tweet in tweepy.Cursor(api.user_timeline, screen_name=username, tweet_mode='extended').items(num_tweets):
            tweets.append(tweet.full_text)
    except tweepy.TweepyException as e:
        print(f"Error fetching tweets: {e}")
    
    return tweets

# Example usage:
username = 'BBCBreaking'  # Replace with any Twitter username
tweets = fetch_tweets(username, num_tweets=200)
print(f"Fetched {len(tweets)} tweets from {username}")


Error fetching tweets: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product
Fetched 0 tweets from BBCBreaking


In [38]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\velpu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\velpu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\velpu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
def scrape_tweets(username):
    url = f"https://x.com/{username}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tweets = [p.text for p in soup.find_all('p', class_='tweet-text')]
    return tweets

def fetch_news(api_key, query):
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey={api_key}"
    response = requests.get(url).json()
    articles = [article['content'] for article in response['articles'] if article['content']]
    return articles

In [5]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

In [3]:
def apply_lda(docs, num_topics=10):
    texts = [clean_text(doc).split() for doc in docs]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    return lda, dictionary, corpus

In [2]:

# Step 4: Generate Word Cloud
def create_word_cloud(lda_model, dictionary):
    topics = lda_model.show_topics(formatted=False)

    for i, topic in topics:
        topic_words = {
        dictionary[word_id]: prob  # Mapping the word (from dictionary) to its probability
        for word_id, prob in topic  # Loop through each (word_id, probability) pair in the topic
            if word_id in dictionary.token2id.values()  # Only include word IDs that exist in the dictionary
        }
        if topic_words:
            wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(topic_words)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.title(f'Topic {i+1}')
            plt.show()
        

# Step 5: Interactive Word Cloud with Dash
def display_interactive_word_cloud(lda_model, dictionary, corpus, texts):
    app = Dash(__name__)

    # Extract topic data
    topic_data = []
    topics = lda_model.show_topics(formatted=False)
    for i, topic in topics:
        topic_words = {dictionary[word_id]: prob for word_id, prob in topic if word_id in dictionary.token2id.values()}
        topic_data.append((i, topic_words))

    def generate_word_cloud(topic_words):
        wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(topic_words)
        fig = px.imshow(wordcloud.to_array())
        fig.update_layout(coloraxis_showscale=False, title="Word Cloud")
        return fig

    app.layout = html.Div([
        dcc.Dropdown(
            id='topic-dropdown',
            options=[{'label': f'Topic {i+1}', 'value': i} for i, _ in enumerate(topics)],
            value=0
        ),
        dcc.Graph(id='word-cloud'),
        dcc.Graph(id='related-tweets')
    ])

    @app.callback(
        Output('word-cloud', 'figure'),
        Input('topic-dropdown', 'value')
    )
    def update_word_cloud(topic_index):
        topic_words = topic_data[topic_index][1]
        return generate_word_cloud(topic_words)

    @app.callback(
        Output('related-tweets', 'figure'),
        Input('word-cloud', 'clickData'),
        Input('topic-dropdown', 'value')
    )
    def display_related_tweets(click_data, topic_index):
        if click_data:
            word = click_data['points'][0]['label']
            related_docs = [text for text, bow in zip(texts, corpus) if any(word in dictionary[id] for id, _ in bow)]
            fig = go.Figure(data=[go.Table(
                header=dict(values=['Related Tweets']),
                cells=dict(values=[related_docs])
            )])
            return fig
        return go.Figure()

    app.run_server(debug=True)


In [6]:
if __name__ == '__main__':
    # Example usage:
    tweets = scrape_tweets('technology')
    print(tweets)
    api_key = '9a2e8fd7507040aab3d87bb793f5a6cd'
    articles = fetch_news(api_key, 'technology')
    print(articles)
    lda_model, dictionary, corpus = apply_lda(articles)
    create_word_cloud(lda_model, dictionary)
    display_interactive_word_cloud(lda_model, dictionary, corpus, articles)

NameError: name 'requests' is not defined